In [1]:
# 使用gps提取的特征完成lightGBM训练；
# 由于gps文件过大，使用华为云不采样估计需要处理几十天。
# 且由于时间限制，设置采样率为1/400，和1/200运行两次；由于丢失了绝大部分数据，对成绩基本没有提升。
# 1/200的相对1/400的在和TTI的相关系数上有10倍的提升，虽然仍然不够，但此方法是有用的，只是时间和算力上不允许。
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")
%cd E:\python_project\MLPS7

e:\python_project\MLPS7


In [12]:
# 需要修改下下面的文件名
df3 = pd.read_csv('train_TTI_fillna_carspeed_count.csv')
df3['time'] = pd.to_datetime(df3['time'])
#df3['hour']=[x.hour+x.minute/60 for x in df3['time']] 
print(df3.corr())

id_road       TTI     speed  carspeed     count
id_road   1.000000  0.263261 -0.677130  0.004982  0.035215
TTI       0.263261  1.000000 -0.592766  0.002093 -0.046711
speed    -0.677130 -0.592766  1.000000 -0.003638 -0.052217
carspeed  0.004982  0.002093 -0.003638  1.000000 -0.002583
count     0.035215 -0.046711 -0.052217 -0.002583  1.000000


In [13]:
# 训练集数据处理1: 6+6+3+6，6 TTI, 3 speed, 3 TTI, 6 carSpeed(想要预测的值)
# 将来如果得到预测集的3 speed，则可以变为6+6+3+6
id_road = df3['id_road'].unique()   # 共12条road
period1 = pd.date_range('2019-01-01 00:00:00', '2019-03-31 23:50:00', freq='10T')
period2 = pd.date_range('2019-10-01 00:00:00', '2019-12-21 23:50:00', freq='10T')
period = period1.append(period2)
index = pd.MultiIndex.from_product([id_road, period1.append(period2)])
columnsList=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5',
            'speed0','speed1','speed2','speed3','speed4','speed5',
            'TTI6','TTI7','TTI8',
            'carSpeed0','carspeed1','carSpeed2','carSpeed3','carSpeed4','carSpeed5','aveCarSpeed',
            'count0','count1', 'count2','count3', 'count4','count5','aveCount']
df4 = pd.DataFrame(0, index=index, columns=columnsList)
for name, group in df3.groupby('id_road'):
    # df3行数：6*24*172*12, 解释：每天6*24条记录，172天，12条road
    matrix = np.zeros((6*24*172, 6+6+3+7+7))
    tti = group['TTI'].values
    speed = group['speed'].values
    carSpeed=group['carspeed'].values
    count=group['count'].values
    i = 6
    for j in range(6, 6*24*90-2):   # 前90天
        matrix[i,0:6] = tti[j-6:j]
        matrix[i,6:12] = speed[j-6:j]
        matrix[i,12:15] = tti[j:j+3]
        matrix[i,15:21] = carSpeed[j-6:j]
        matrix[i,21]=np.sum(carSpeed[j-6:j])/6
        matrix[i,22:28]=count[j-6:j]
        matrix[i,28]=np.sum(count[j-6:j])/6
        i += 1
    i += 8
    for j in range(6*24*90+6, 6*24*172-2):  # 后172-90天
        matrix[i,0:6] = tti[j-6:j]
        matrix[i,6:12] = speed[j-6:j]
        matrix[i,12:15] = tti[j:j+3]
        matrix[i,15:21] = carSpeed[j-6:j]
        matrix[i,21]=np.sum(carSpeed[j-6:j])/6
        matrix[i,22:28]=count[j-6:j]
        matrix[i,28]=np.sum(count[j-6:j])/6
        i += 1
    df4.loc[name] = matrix
df4.index.names = ['id_road', 'time']
df4=df4.loc[df4['TTI0']>0]
df4.to_csv('train_TTI_6+6+3+7+7.csv')
#df4

In [14]:
# 训练集数据处理2: 添加特征
import calendar
df5 = pd.read_csv('train_TTI_6+6+3+7+7.csv')
df5['time']=pd.to_datetime(df5['time'])
# 添加是否放假的特征
month_day=[[x.month,x.day] for x in df5['time']]
weekday=[calendar.weekday(x.year,x.month,x.day) for x in df5['time']]
holidayList=[[1,1],[2,4],[2,5],[2,6],[2,7],[2,8],[2,9],[2,10],[10,1],[10,2],[10,3],[10,4],[10,5],[10,6],[10,7]]
workdayList=[[2,2],[2,3],[10,12]]

isHoliday=np.zeros(np.size(weekday))
for i in range(np.size(weekday)):
    if month_day[i] in holidayList:
        isHoliday[i]=1.0
    elif weekday[i]>=5:
        isHoliday[i]=1.0
    else:
        isHoliday[i]=0.0
    if month_day[i] in workdayList:
        isHoliday[i]=0.0
df5['isHoliday']=isHoliday
df5['weekday']=weekday
df5.to_csv('train_TTI_6+6+3+7+7_more.csv',index=0)

In [23]:
# 相关系数
df9=df5
df9['TTI']=df9['TTI6']
print(df9.corr())


id_road      TTI0      TTI1      TTI2      TTI3      TTI4  \
id_road      1.000000e+00  0.263144  0.263115  0.263102  0.263123  0.263155   
TTI0         2.631442e-01  1.000000  0.890297  0.782578  0.710035  0.658986   
TTI1         2.631151e-01  0.890297  1.000000  0.890323  0.782670  0.710126   
TTI2         2.631023e-01  0.782578  0.890323  1.000000  0.890336  0.782680   
TTI3         2.631234e-01  0.710035  0.782670  0.890336  1.000000  0.890337   
TTI4         2.631548e-01  0.658986  0.710126  0.782680  0.890337  1.000000   
TTI5         2.631735e-01  0.618431  0.659076  0.710140  0.782680  0.890334   
speed0      -6.770207e-01 -0.592764 -0.566761 -0.537874 -0.513184 -0.491079   
speed1      -6.770256e-01 -0.567401 -0.592747 -0.566739 -0.537884 -0.513200   
speed2      -6.770272e-01 -0.538816 -0.567394 -0.592733 -0.566751 -0.537899   
speed3      -6.770344e-01 -0.515558 -0.538835 -0.567388 -0.592744 -0.566765   
speed4      -6.770464e-01 -0.495206 -0.515583 -0.538832 -0.567398 -0.5

In [16]:
# 测试集数据处理0:  将toPredict_nolabel.csv和toPredict_train_TTI.csv合并为一个直接可以给模型调用的文件
# 列向量：id_road	TTI	speed	time	id_sample   carspeed    count   index
# 需要先填充index: 0到n-1
data_test = pd.read_csv('toPredict_Xy+6+6.csv')
data_test['time']=pd.to_datetime(data_test['time'])
data_test['index']=data_test.index
data_test['index']=[int(x/9) for x in data_test['index']]
Xy_test_new=pd.DataFrame(columns=['id_road','time','TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','id_sample0','id_sample1','id_sample2',
'carSpeed0','carspeed1','carSpeed2','carSpeed3','carSpeed4','carSpeed5','aveCarSpeed',
'count0','count1', 'count2','count3', 'count4','count5','aveCount'])
for index,group in data_test.groupby('index'):
    ttl=group['TTI'].values
    id_road=group.iloc[0,0]
    speed=group['speed'].values
    id_sample=group['id_sample'].values
    carSpeed=group['carspeed'].values
    aveCarSpeed=np.sum(carSpeed[0:6])/6
    count=group['count'].values
    aveCount=np.sum(count[0:6])/6
    time=group['time'].values
    Xy_test_new.loc[len(Xy_test_new)]=[id_road,time[6],ttl[0],ttl[1],ttl[2],ttl[3],ttl[4],ttl[5],speed[0],speed[1],speed[2],speed[3],speed[4],speed[5],id_sample[6],id_sample[7],id_sample[8],carSpeed[0],carSpeed[1],carSpeed[2],carSpeed[3],carSpeed[4],carSpeed[5],aveCarSpeed,count[0],count[1],count[2],count[3],count[4],count[5],aveCount]
Xy_test_new.to_csv('test_6+6+3+7+7.csv',index=0)

In [17]:
# 测试集数据处理1: 添加特征
import calendar
Xy_test_new = pd.read_csv('test_6+6+3+7+7.csv')
Xy_test_new['time']=pd.to_datetime(Xy_test_new['time'])
# 添加是否放假的特征
month_day=[[x.month,x.day] for x in Xy_test_new['time']]
weekday=[calendar.weekday(x.year,x.month,x.day) for x in Xy_test_new['time']]
holidayList=[[1,1],[2,4],[2,5],[2,6],[2,7],[2,8],[2,9],[2,10],[10,1],[10,2],[10,3],[10,4],[10,5],[10,6],[10,7]]
workdayList=[[2,2],[2,3],[10,12]]

isHoliday=np.zeros(np.size(weekday))
for i in range(np.size(weekday)):
    if month_day[i] in holidayList:
        isHoliday[i]=1.0
    elif weekday[i]>=5:
        isHoliday[i]=1.0
    else:
        isHoliday[i]=0.0
    if month_day[i] in workdayList:
        isHoliday[i]=0.0
Xy_test_new['isHoliday']=isHoliday
Xy_test_new['weekday']=weekday
Xy_test_new.to_csv('test_6+6+3+7+7_more.csv',index=0)

In [29]:
df5 = pd.read_csv('train_TTI_6+6+3+7+7_more.csv')
df5['time']=pd.to_datetime(df5['time'])
df5['month']=[x.month for x in df5['time']]
data_train = df5
data_train['hour']=[x.hour+x.minute/60 for x in data_train['time']]    # 只保留小时和分
data_train=data_train.loc[data_train['hour']>=7.5]
data_train=data_train.loc[data_train['hour']<=19]
#display(data_train)
Xy_test=pd.read_csv('test_6+6+3+7+7_more.csv')
Xy_test['time']=pd.to_datetime(Xy_test['time'])
Xy_test['hour']=[x.hour+x.minute/60 for x in Xy_test['time']]    # 只保留小时和分
y_predict=np.zeros(Xy_test.shape[0]*3)
for id_road,group in Xy_test.groupby('id_road'):
    group_train=data_train.loc[data_train['id_road']==id_road]
    X_columns_list=['TTI0','TTI1','TTI2','TTI3','TTI4','TTI5','speed0','speed1','speed2','speed3','speed4','speed5','aveCarSpeed','aveCount','hour','isHoliday']
    y_columns_list=['TTI6','TTI7','TTI8']
    X=group_train[X_columns_list].values
    y=group_train[y_columns_list].values
    y0=y[:,0]
    y1=y[:,1]
    y2=y[:,2]
    model0= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model1= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model2= LGBMRegressor(num_leaves=31,max_depth=10,learning_rate=0.02,n_estimators=500,min_child_samples=22)
    model0.fit(X, y0)
    model1.fit(X, y1)
    model2.fit(X, y2)
    id_sample_columns_list=['id_sample0','id_sample1','id_sample2']
    X_test=group[X_columns_list].values
    id_sample=group[id_sample_columns_list].values
    y0_predict=model0.predict(X_test)
    y1_predict=model1.predict(X_test)
    y2_predict=model2.predict(X_test)
    
    for i in range(np.size(y0_predict)):
        y_predict[int(id_sample[i,0])]=y0_predict[i]
        y_predict[int(id_sample[i,1])]=y1_predict[i]
        y_predict[int(id_sample[i,2])]=y2_predict[i]
y_predict

array([1.07042657, 1.08290749, 1.1783111 , ..., 1.62108211, 1.58583826,
       1.57086469])

In [30]:
# 输出y_predict
out=np.zeros([len(y_predict),2])
for i in range(len(y_predict)):
    out[i,0]=i
    out[i,1]=y_predict[i]
out=pd.DataFrame(out,columns=['id_sample','TTI'])
out.to_csv('submit18_gps删除.csv',mode='w',index=False,header=True,encoding= 'utf-8')